## Graph Neural Networks with Pytorch
## Target: Graph Attention Networks
- Original Paper: https://arxiv.org/abs/1710.10903
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gat.py

In [1]:
import os
import sys
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

sys.path.append('../')
from utils import *
logger = make_logger(name='gcn_logger')

# Load Cora Dataset
dataset = 'Cora'
path = os.path.join(os.getcwd(), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [2]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(
            in_channels=in_channels, out_channels=8, heads=8, dropout=0.6)
        self.conv2 = GATConv(
            in_channels=8*8, out_channels=out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)

**GATConv**의 구조를 살펴본다.  

|기호|설명|
|:--------:|:--------:|
|$N$| Node 수 |
|$F$| Node의 Feature 수 |
|$F^{\prime}$| Hidden Layer 길이 |
|$W$| Trainable Parameter |
|$\vec{a}^{T}$| Trainable Parameter |
  
  
아래와 같이 표현되는 Node Feature Matrix가 존재한다고 해보자,  

$$ \mathbf{h} = \{ \vec{h}_1, \vec{h}_2, ..., \vec{h}_N  \}  $$  

위 행렬의 초기 형태는 $(N, F)$ 이고, GAT Convolutional Layer를 통과한 후에는 $(N, F^{\prime})$ 의 형태를 취하게 될 것이다.  

$W = (F^{\prime}, F), \mathbf{a} = (2F^{\prime}, 1)$ 의 shape을 갖고 있을 때 **Normalized Attention Score**는 아래와 같이 계산한다.  

$$ \alpha_{ij} = \frac {exp (LeakyRELU ( \vec{a}^T [ \mathbf{W} \vec{h}_i \vert \mathbf{W} \vec{h}_j ] )) } { \Sigma_{k \in N_i} exp ( LeakyRELU ( \vec{a}^T [ \mathbf{W} \vec{h}_i \vert \mathbf{W} \vec{h}_k ] ) ) } $$  

위 식은 Node $i$ 에 대해 Node $j$ 의 Feature가 갖는 **중요도**를 의미한다. 이 때 $j$ 는 모든 Node를 의미하는 것은 아니고 $N_i$ 즉, Node $i$ 의 이웃에 대해서만 계산하게 된다.  

이렇게 계산된 **Attention Score**는 아래와 같이 Node $i$의 이웃의 중요도를 결정하여 Input 데이터를 재정의하게 된다.  

$$ \vec{h}^{\prime}_i = \sigma( \Sigma_{j \in N_i} \alpha_{ij} \mathbf{W} \vec{h}_j ) $$  


**GATConv**에서 여러 인자를 활용할 수 있다. `in_channels`와 `out_channels` 인자는 Input과 Output의 마지막 Shape을 의미한다.  

`heads`는 Multi-head Attention Loop를 몇 번 할 것인가를 의미하며, 기본 값은 1이다.  

`concat`의 기본 값은 True이며, 만약 False로 설정할 경우 Multi-head Attention이 concatenated되는 것이 아니라 평균화된다.  

`negative_slope`는 LeakyReLU 함수의 slope 값을 의미한다.  

`forward` 할 때는 edge_index를 요구하며, `return_attenion_weights`를 True로 설정할 경우 Attention Weights를 포함한 튜플 형태의 결과 값을 반환할 것이다.  

추가적인 참고자료가 필요하다면 [공식 문서](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GATConv)를 확인해도 좋고, [GAT 논문 리뷰 글](https://greeksharifa.github.io/machine_learning/2021/05/29/GAT/#21-graph-attentional-layer)을 참고해도 좋다.  

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

logger.info(f"GAT In: {dataset.num_features}, Out: {dataset.num_classes}")
model = GAT(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

2021-08-09 13:00:45,077 - gcn_logger - GAT In: 1433, Out: 7


In [4]:
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs

In [5]:
for epoch in range(1, 201):
    train(data)
    train_acc, val_acc, test_acc = test(data)
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2786, Val: 0.2500, Test: 0.2450
Epoch: 002, Train: 0.4857, Val: 0.3660, Test: 0.3850
Epoch: 003, Train: 0.5214, Val: 0.4060, Test: 0.4140
Epoch: 004, Train: 0.5929, Val: 0.4200, Test: 0.4110
Epoch: 005, Train: 0.6571, Val: 0.4680, Test: 0.4500
Epoch: 006, Train: 0.7214, Val: 0.5320, Test: 0.5210
Epoch: 007, Train: 0.8429, Val: 0.6280, Test: 0.6500
Epoch: 008, Train: 0.8571, Val: 0.6880, Test: 0.7060
Epoch: 009, Train: 0.8786, Val: 0.7020, Test: 0.7260
Epoch: 010, Train: 0.8857, Val: 0.7100, Test: 0.7300
Epoch: 011, Train: 0.8929, Val: 0.7080, Test: 0.7300
Epoch: 012, Train: 0.8857, Val: 0.7240, Test: 0.7370
Epoch: 013, Train: 0.8929, Val: 0.7340, Test: 0.7490
Epoch: 014, Train: 0.9214, Val: 0.7580, Test: 0.7650
Epoch: 015, Train: 0.9286, Val: 0.7580, Test: 0.7560
Epoch: 016, Train: 0.9214, Val: 0.7560, Test: 0.7550
Epoch: 017, Train: 0.9357, Val: 0.7580, Test: 0.7620
Epoch: 018, Train: 0.9357, Val: 0.7660, Test: 0.7740
Epoch: 019, Train: 0.9571, Val: 0.7780, Test: 